In [ ]:
library(tidyverse)

In [ ]:
#t = "C10AA"
#t = "C10AB"
#t = "C10AX06"
t = "C10AX09"

#t = "C03"
#t = "C07"
#t = "C08"
#t = "C09"
#t = "PA"

## Load PGS

In [ ]:
data = read.table(
#    paste0("score_lipidaemiadrug/1/score.", t, ".chr1.sscore"), header=TRUE, comment.char="")
    paste0("score.20250929/1/score.", t, ".chr1.sscore"), header=TRUE, comment.char="")
#data$pgs = data$SCORE1_AVG
#data$pgs = data$NAMED_ALLELE_DOSAGE_SUM * data$SCORE1_AVG
#data$pgs = data$ALLELE_CT * data$SCORE1_AVG
colnames(data)[colnames(data)=="SCORE1_SUM"] = "pgs"

In [ ]:
for (i in 2:22) {
#for (i in c(2:8, 10:13, 15, 18)) { #PA
    foo = read.table(
#        paste0("score_lipidaemiadrug/", i, "/score.", t, ".chr", i, ".sscore"), header=TRUE, comment.char="")
        paste0("score.20250929/", i, "/score.", t, ".chr", i, ".sscore"), header=TRUE, comment.char="")
    print(identical(data$X.FID, foo$X.FID))
    #foo$pgs = foo$SCORE1_AVG
    #foo$pgs = foo$NAMED_ALLELE_DOSAGE_SUM * foo$SCORE1_AVG
    #foo$pgs = foo$ALLELE_CT * foo$SCORE1_AVG
    colnames(foo)[colnames(foo)=="SCORE1_SUM"] = "pgs"
    data[, 3:6] = data[, 3:6] + foo[, 3:6]
}

In [ ]:
colnames(data)[1] = 'FID'

In [ ]:
head(data)

In [ ]:
table(data$ALLELE_CT)

## Load basal score

In [ ]:
foo = read.table(
#    "aou_lipidaemiadrug_covariates.txt",
#    "aou_lipidaemiadrug_withC10B_covariates.txt",
    "aou_hypertensiondrug_covariates.txt",
    header=TRUE)

In [ ]:
data = cbind(data, foo[match(data$FID, foo$FID), -c(1:2)])

In [ ]:
colnames(data)

In [ ]:
foo = read.table(
    paste0("coeff_basal.", t, ".txt"),
    header=FALSE)

In [ ]:
data$basal = cbind(1, as.matrix(data[, c("sexM", "age", "bmi")])) %*% foo[, 2]

In [ ]:
data$basalpgs = data$basal + data$pgs

## Load phenotype, PC1

In [ ]:
foo = read.table(
#    "aou_lipidaemiadrug_BT_4months.txt",
    "aou_lipidaemiadrug_withC10B_BT_3months.txt",
#    "aou_hypertensiondrug_BT_4months.txt",
#    "aou_hypertensiondrug_onlyaffected_BT_3months.txt",
    header=TRUE)
data$trait = factor(foo[match(data$FID, foo$FID), t])

In [ ]:
foo = read.table(
    "aou_lipidaemiadrug_withC10B_covariates.txt",
#    "aou_hypertensiondrug_onlyaffected_covariates.txt",
    header=TRUE)
data$PC1 = foo$PC1[match(data$FID, foo$FID)]

In [ ]:
foo = read.table(
#    "aldosterone_srWGS.aldosterone.nodrug.eur.txt",
#    "potassiumserumplasma_srWGS.potassium.eur.txt",
    "DBPSBP_srWGS.SBP.nodrug.eur.txt",
    header=TRUE)
print(dim(foo))
foo = foo %>%
  group_by(person_id) %>%
  summarize(PA = mean(value_as_number), .groups="drop") %>% #In reality aldosterone, not the dicotomous PA
  rename(FID = person_id)
foo = as.data.frame(foo)
print(dim(foo))
data$trait = foo[match(data$FID, foo$FID), t]

## Post-processing

In [ ]:
table(is.na(data$PC1))

In [ ]:
# this cohort & eur
data = data[!is.na(data$PC1), ]

In [ ]:
data = data[!is.na(data$trait), ]

In [ ]:
data$pgsdecile = cut(data$pgs, quantile(data$pgs, seq(0, 1, 0.1)), include.lowest=TRUE)

In [ ]:
data$pgsquartile = cut(data$pgs, quantile(data$pgs, seq(0, 1, 0.25)), include.lowest=TRUE)

## Plot

PAPGS vs aldosteron, potassium

In [ ]:
plotdata = data %>%
select(pgsdecile, trait) %>%
group_by(pgsdecile) %>%
summarize(proportion = mean(trait=="1"))

In [ ]:
p =
ggplot(data=plotdata) +
geom_bar(aes(x=pgsdecile, y=proportion), stat='identity') +
theme(axis.text.x = element_text(angle=90, hjust=1, vjust=.5)) +
xlab("Deciles of PGS") +
ylab("Proportion taking drug") +
ggtitle(t)
ggsave(p, file="Rplot.pdf")
show(p)

In [ ]:
p = ggplot(data=data) +
  geom_boxplot(aes(x=pgsquartile, y=trait)) +
#  scale_y_log10() + #Aldosterone
  xlab("PA PGS") +
#  ylab("Aldosterone [ng/dL]")
#  ylab("Potassium [mmol/L]")
  ylab("SBP [mmHg]")
ggsave(p, file="Rplot.pdf")
p
#summary(lm(log10(trait) ~ pgs, data=data))
summary(lm(trait ~ pgs, data=data))

## Statistical tests

In [ ]:
table(data$trait)

In [ ]:
x = table(data$pgsdecile,data$trait)
x

In [ ]:
x[, 2] / rowSums(x)

In [ ]:
a0 = glm(trait ~ pgs, data=data, family=binomial())
summary(a0)

In [ ]:
summary(a0)$coefficient

In [ ]:
anova(a0)[2,2]/anova(a0)[1,4]

P for AUC > 0.5

In [ ]:
x = wilcox.test(data$pgs[data$trait==1], data$pgs[data$trait==0], alternative="greater")
x

In [ ]:
x$p.value

In [ ]:
a0 = glm(trait ~ basal, data=data, family=binomial())
summary(a0)

In [ ]:
a0 = glm(trait ~ basalpgs, data=data, family=binomial())
summary(a0)

In [ ]:
a0 = glm(trait ~ ., data=data[, -c(1:5, 8, 11)], family=binomial())
summary(a0)

In [ ]:
a0 = glm(trait ~ pgsdecile, data=data, family=binomial())
summary(a0)

In [ ]:
library("pROC")

In [ ]:
roc(response=data$trait, predictor=data$pgs, direction="<")